In [ ]:
import sys
import pandas as pd
from lxml import etree
import re
from thefuzz import fuzz

print(sys.executable)

## Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [ ]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''
    
    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'
    
    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)    
    else:
        localisationca = 'none'
    
    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)            
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)
        
    else: 
        localisationco = 'none'
    
    #on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)    

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

In [ ]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco']]

df.iloc[1500:1510, :]

In [ ]:
# Chargement du référentiel des communes
# NB. pour Margot fields (on économise de la mémoire) + dtype
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

In [ ]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

## Liage des articles de type commune

In [ ]:
# df des communes liées (pour décomptes et concaténation dans l’output final relu par OC et SN)
linked_communes_df = pd.DataFrame()

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[1500:1510, :]
#communes_df.shape

### Exact match method

#### With dpt

In [ ]:
# liage des communes avec dpt de localisation (dpt/exact match)
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
communes_exact_dpt_df.head(5)
#communes_exact_dpt_df.shape

In [ ]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/exact match)
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on = 'NCCENR')
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# Test
communes_exact_nodpt_df
linked_communes_df.shape

In [ ]:
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False, subset=['article_id']), :]

In [ ]:
linked_communes_df['article_id']

### Fuzzy method

In [ ]:
import difflib

#### With dpt

In [ ]:
# liage des communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_communes_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: difflib.get_close_matches(x['vedette'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR']),
    axis=1
)
# sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda d: len(d)) > 0]
# list2string (pour dedoublonner et imprimer)
# https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
communes_fuzzy_dpt_df['insee_code'] = [' ; '.join(map(str, l)) for l in communes_fuzzy_dpt_df['insee_code']]

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_fuzzy_dpt_df]).drop_duplicates()

In [ ]:
# Tests
#communes_fuzzy_dpt_df.head(20)
#linked_communes_df.shape
linked_communes_df

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_communes_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette'].apply(
    lambda x: difflib.get_close_matches(x, main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)]['NCCENR']))

communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda d: len(d)) > 0]
communes_fuzzy_nodpt_df

# https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
communes_fuzzy_nodpt_df['insee_code'] = [' ; '.join(map(str, l)) for l in communes_fuzzy_nodpt_df['insee_code']]

# actualiser le df des liages réalisés
linked_communes_df = pd.concat([linked_communes_df, communes_fuzzy_nodpt_df]).drop_duplicates()

In [ ]:
# Tests
#communes_fuzzy_nodpt_df
communes_fuzzy_nodpt_df.shape
linked_communes_df.shape

## Liage des communes de localisation

### Exact match method

#### With dpt

#### Without dpt

### Fuzzy match method

#### With dpt

#### Without dpt

## Rapport et analyse

## Export

In [ ]:
# MARGOT – REVOIR
# Liage des exact-matches ('vedette' == 'NCCENR' du référentiel des communes)

# merge sur dpt_code/DEP_id – ALGO très lourd : produire un df de toutes le combinaisons possibles article/communes par dpt 
# TODO: définir un ALGO plus light ! – dédoublonner d’emblée ?
commune_merged_df = pd.merge(
    commune_df,
    main_insee_commune,
    left_on='dpt_code',
    right_on='DEP_id',
    how='left')
#commune_merged_df.head(20)

# Sélectionner les lignes où 'vedette' == 'NCCENR'
matching_rows = commune_merged_df.loc[commune_merged_df['vedette'] == commune_merged_df['NCCENR']]
# ajout du liage aux df des communes à lier – 'none' sinon
commune_df['insee_code'] = commune_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['vedette']]['insee_code'].values) > 0 else None, axis=1)
commune_df['insee_code'] = commune_df['insee_code'].fillna('none')

commune_df.head(10)

In [ ]:
#tous nos article_id qui correspondent à un code insee ; on peut donc en conclure que ce sont effectivement des 
#communes.
# ICI les seules communes liées par exact match… On aurait pu en faire le df a priori au merge précédent (NCCENR ==vedette)…

commune_insee = commune_df[commune_df['insee_code'] != 'none']
print(commune_insee)

#on a donc exactement 2357 matches.

In [ ]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

commune_insee['method'] = 'exact'
commune_insee.loc[commune_insee['dpt_code'] == 'none', 'method'] = 'nodpt'

print(commune_insee)

In [ ]:
#les articles qui du coup ne seraient apparemment pas des communes.

commune_noinsee = commune_df.loc[commune_df['insee_code'] == 'none']
print(commune_noinsee)

#nous en avons donc 391.

In [ ]:
# Définition d'une fonction pour le fuzzy join

def fuzzy_join(row, distance):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du localisationco_pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à distance
    filtered = main_dep[scores >= distance]
    # Ajout de la colonne insee_code dans le dataframe localisationco_pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'
    
# Application de la fonction sur chaque ligne du dataframe localisationco_pascommune
commune_noinsee['insee_code'] = commune_noinsee.apply(fuzzy_join, distance=90, axis=1)

In [ ]:
print(commune_noinsee)

In [ ]:
#on veut savoir combien ont été joined par le fuzzy

commune_fuzzy = commune_noinsee[commune_noinsee['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy

commune_fuzzy['method'] = 'fuzzy'

print(commune_fuzzy)

In [ ]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

commune_fuzzynull = commune_noinsee[commune_noinsee['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy.
#on différencie bien sûr ceux n'ayant simplement pas de dpt_code

commune_fuzzynull['method'] = 'nulle'
print(commune_fuzzynull)

In [ ]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

commune_final_df = pd.concat([commune_insee, commune_fuzzy, commune_fuzzynull], axis=0)
commune_final_df = commune_final_df.sort_values('method', ascending=True)

print(commune_final_df)

In [ ]:
commune_final_df['origine'] = 'sansloc'

## Liage des communes de localisation (article dans une commune)

Il s’agit des lieux à l’échelle infra-communale. Ces lieux sont localisés dans une commune (`localisationco`)
…

In [ ]:
# Liage des articles localisés dans une commune

localisationco_df = df[df['localisationco'] != 'none']
localisationco_df['dpt_code'] = localisationco_df['dpt_code'].apply(lambda x: 'DEP_' + str(x) if x != 'none' else x)
localisationco_df.head(5)

In [ ]:
#On lie notre dataframe au référentiel insee avec de l'exact match.

main_insee_commune = pd.read_csv("main_insee_commune.tsv", delimiter='\t')

# on utilise merge pour le liage avec dpt_code et dep_id
localisationco_merged_df = pd.merge(localisationco_df, main_insee_commune, left_on='dpt_code', right_on='DEP_id', how='left')

# on sélectionne les lignes où la colonne 'localisationco' correspond à la colonne 'NCCENR' de main_insee_commune
# exact match
matching_rows = localisationco_merged_df.loc[localisationco_merged_df['localisationco'] == localisationco_merged_df['NCCENR']]

# on crée une colonne 'insee_code' dans le dataframe localisationco_df avec les codes correspondants à partir de main_insee_commune
localisationco_df['insee_code'] = localisationco_df.apply(lambda row: main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values[0] if len(main_insee_commune.loc[main_insee_commune['NCCENR'] == row['localisationco']]['insee_code'].values) > 0 else None, axis=1)
localisationco_df['insee_code'] = localisationco_df['insee_code'].fillna('none')


In [ ]:
localisationco_df.head(5)

In [ ]:
# vérifier les localisationco_communes de localisation liées sur exact match

localisationco_commune = localisationco_df[localisationco_df['insee_code'] != 'none']
localisationco_commune.head(10)

#on a donc exactement 556 matches.

In [ ]:
print(localisationco_commune)

In [ ]:
#modification de notre dataframe: ajout d'une colonne method qui nous permet d'identifier la méthode utilisée pour le
#liage

localisationco_commune['method'] = 'exact'
# marquer les liages exécutés sans possibilité de désambiguiser le dpt (pour validation manuelle a posteriori)
localisationco_commune.loc[localisationco_commune['dpt_code'] == 'none', 'method'] = 'nodpt'

print(localisationco_commune)

In [ ]:
#les articles qui du coup n'ont pas de liage. on fera le fuzzy pour eux.

localisationco_pascommune = localisationco_df.loc[localisationco_df['insee_code'] == 'none']
print(localisationco_pascommune)

#nous en avons donc 124.

In [ ]:
# Définition d'une fonction pour le fuzzy join

def fuzzy_join(row, distance):
    # Filtre sur les lignes avec les mêmes départements
    main_dep = main_insee_commune[main_insee_commune['DEP_id'] == row['dpt_code']]
    # Calcul de la distance Levenshtein entre la vedette du localisationco_pascommune et les NCCENR de chaque ligne de main_dep
    scores = main_dep['NCCENR'].apply(lambda x: fuzz.token_sort_ratio(x, row['vedette']))
    # Récupération des lignes avec une distance Levenshtein inférieure ou égale à distance
    filtered = main_dep[scores >= distance]
    # Ajout de la colonne insee_code dans le dataframe localisationco_pascommune
    if not filtered.empty:
        insee_code = filtered['insee_code'].values[0]
        if pd.isnull(insee_code):
            return 'none'
        else:
            return insee_code
    else:
        return 'none'
    
# Application de la fonction sur chaque ligne du dataframe localisationco_pascommune
localisationco_pascommune['insee_code'] = localisationco_pascommune.apply(fuzzy_join, distance=90, axis=1)

In [ ]:
print(localisationco_pascommune)

In [ ]:
#on veut savoir combien ont été joined par le fuzzy_localisationco

fuzzy_localisationco = localisationco_pascommune[localisationco_pascommune['insee_code'] != 'none']

# on ajoute une colonne method pour indiquer que le join a été réalisé par fuzzy_localisationco

fuzzy_localisationco['method'] = 'fuzzy'


print(fuzzy_localisationco)

In [ ]:
# maintenant, on s'intéresse aux localisationco qui n'ont toujours pas de liage.

fuzzynull_localisationco = localisationco_pascommune[localisationco_pascommune['insee_code'] == 'none']

#idem, on ajoute la méthode pour savoir que ces derniers n'ont été matchés ni avec le exact ni avec le fuzzy

fuzzynull_localisationco['method'] = 'nulle'
print(fuzzynull_localisationco)

In [ ]:
#il faut maintenant concaténer nos 3 dataframes, en respectant l'ordre original des article_id

localisationco_final_df = pd.concat([localisationco_commune, fuzzy_localisationco, fuzzynull_localisationco], axis=0)
localisationco_final_df = localisationco_final_df.sort_values('method', ascending=True)

print(localisationco_final_df)

In [ ]:
localisationco_final_df['origine'] = 'avecloc'

## Fusion et ordonnancement des liages

In [ ]:
# Créer une colonne de correspondance pour les deux dataframes
commune_final_df['corresponding_value'] = commune_final_df['vedette']
localisationco_final_df['corresponding_value'] = localisationco_final_df['localisationco']

# Concaténer les deux dataframes
merged_df = pd.concat([commune_final_df, localisationco_final_df], sort=False)

# Trier les valeurs par ordre de la colonne "corresponding_value" puis par ordre de la colonne "article_id"
merged_df.sort_values(by=['corresponding_value', 'article_id'], inplace=True)

# Supprimer la colonne "corresponding_value"
merged_df.drop(columns=['corresponding_value'], inplace=True)

print(merged_df)

In [ ]:
merged_df = pd.merge(merged_df, main_insee_commune[['NCCENR', 'insee_code']], on='insee_code', how='left')
merged_df['NCCENR'] = merged_df['NCCENR'].fillna('none')

In [ ]:
print(merged_df)

In [ ]:
method_counts = merged_df['method'].value_counts()
print(method_counts)

In [ ]:
origine_counts = merged_df['origine'].value_counts()
print(origine_counts)

In [ ]:
# Sauvegarder le dataframe fusionné
merged_df.to_csv('../../utils/liage_po7.csv', index=False)